In [43]:
require("data.table")
require("rpart")
library(mice)
library(imputeTS)
library(randomForest)
require(xgboost)
library(MASS)
library(dplyr)


Attaching package: ‘dplyr’


The following object is masked from ‘package:MASS’:

    select


The following object is masked from ‘package:xgboost’:

    slice


The following object is masked from ‘package:randomForest’:

    combine


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
#Aqui se debe poner la carpeta de la computadora local
setwd("/Users/fran/Documents/Maestria/DM\ en\ Economia\ y\ Finanzas")  #Establezco el Working Directory

In [3]:
dtrain  <- fread("./datasetsOri/paquete_premium_202009.csv")

Genero el modelo, llamando a la funcion rpart

In [13]:
#impute it with mice
# dtrain_imputed = mice(dtrain, m = 3, print=T)

# dtrain_imputed = complete(dtrain_imputed, 1)

dtrain_imputed = na_kalman(dtrain)

Warning message in stats::StructTS(data, ...):
“possible convergence problem: 'optim' gave code = 52 and message ‘ERROR: ABNORMAL_TERMINATION_IN_LNSRCH’”
Warning message in stats::StructTS(data, ...):
“possible convergence problem: 'optim' gave code = 52 and message ‘ERROR: ABNORMAL_TERMINATION_IN_LNSRCH’”
Warning message in stats::StructTS(data, ...):
“possible convergence problem: 'optim' gave code = 52 and message ‘ERROR: ABNORMAL_TERMINATION_IN_LNSRCH’”
Warning message in stats::StructTS(data, ...):
“possible convergence problem: 'optim' gave code = 52 and message ‘ERROR: ABNORMAL_TERMINATION_IN_LNSRCH’”
Warning message in stats::StructTS(data, ...):
“possible convergence problem: 'optim' gave code = 52 and message ‘ERROR: ABNORMAL_TERMINATION_IN_LNSRCH’”
Warning message in stats::StructTS(data, ...):
“possible convergence problem: 'optim' gave code = 52 and message ‘ERROR: ABNORMAL_TERMINATION_IN_LNSRCH’”
Warning message in stats::StructTS(data, ...):
“possible convergence problem

In [19]:
dtrain_imputed_cleaned = dtrain_imputed[, -c('numero_de_cliente', 'foto_mes')]

In [62]:
dtrain_imputed_cleaned_scaled = as.data.frame(scale(dtrain_imputed_cleaned[, -c('clase_ternaria')]))

In [64]:
dtrain_imputed_cleaned_scaled$clase_ternaria = dtrain_imputed_cleaned$clase_ternaria

In [66]:
modelo  <- rpart("clase_ternaria ~ .",
                 data = dtrain_imputed_cleaned_scaled,
                 xval=0,
                 cp=        -0.3, 
                 minsplit=  100,
                 minbucket=   1,
                 maxdepth=   10)
#                 method='poisson')

In [ ]:
# modelo_rf = randomForest(dtrain_imputed[, -c('clase_ternaria')],
#                          as.factor(dtrain_imputed$clase_ternaria),
#                          importance = TRUE,
#                          proximity = TRUE)

In [ ]:
# labels <- dtrain_imputed$clase_ternaria

In [ ]:
# labels <- as.numeric(dtrain_imputed$clase_ternaria) - 1 
# new_tr <- model.matrix(~.+0,data = dtrain_imputed[,-c("clase_ternaria"),with=F]) 

In [ ]:
# dtrain <- xgb.DMatrix(data=new_tr,
#                       label=labels)

In [ ]:
# params <- list(booster = "gbtree", 
#                objective = "multi:softmax", 
#                num_class = 3, 
#                eval_metric = "mlogloss")

In [ ]:
# xgbcv <- xgb.cv(params = params, 
#                 data = dtrain, 
#                 nrounds = 10, 
#                 nfold = 2, 
#                 showsd = TRUE, 
#                 stratified = TRUE, print_every_n = 10, early_stop_round = 20, maximize = FALSE, prediction = TRUE)

In [ ]:
# xgb_model <- xgb.train(params=params, 
#                        data=dtrain, 
#                        nrounds=10, 
# #                        stratified=TRUE,
#                        print_every_n=3,
# #                        early_stop_round=20,
#                        maximize=FALSE)
# #                        prediction=TRUE)

In [67]:
dapply  <- fread("./datasetsOri/paquete_premium_202011.csv")

In [68]:
dapply_imputed = na_kalman(dapply[, -c('clase_ternaria')], 'StructTS')

Warning message in stats::StructTS(data, ...):
“possible convergence problem: 'optim' gave code = 52 and message ‘ERROR: ABNORMAL_TERMINATION_IN_LNSRCH’”
Warning message in stats::StructTS(data, ...):
“possible convergence problem: 'optim' gave code = 52 and message ‘ERROR: ABNORMAL_TERMINATION_IN_LNSRCH’”
Warning message in stats::StructTS(data, ...):
“possible convergence problem: 'optim' gave code = 52 and message ‘ERROR: ABNORMAL_TERMINATION_IN_LNSRCH’”
Warning message in stats::StructTS(data, ...):
“possible convergence problem: 'optim' gave code = 52 and message ‘ERROR: ABNORMAL_TERMINATION_IN_LNSRCH’”
Warning message in stats::StructTS(data, ...):
“possible convergence problem: 'optim' gave code = 52 and message ‘ERROR: ABNORMAL_TERMINATION_IN_LNSRCH’”
Warning message in stats::StructTS(data, ...):
“possible convergence problem: 'optim' gave code = 52 and message ‘ERROR: ABNORMAL_TERMINATION_IN_LNSRCH’”
Warning message in stats::StructTS(data, ...):
“possible convergence problem

In [69]:
dapply_imputed_cleaned = dapply_imputed[, -c('numero_de_cliente', 'foto_mes')]
dtrain_imputed_cleaned_scaled = as.data.frame(scale(dapply_imputed_cleaned[, -c('clase_ternaria')]))

Warning message in `[.data.table`(dapply_imputed_cleaned, , -c("clase_ternaria")):
“column(s) not removed because not found: [clase_ternaria]”


In [ ]:
# xgb_val_preds <- predict(xgb_model, xgb.DMatrix(as.matrix(dapply[, -c('clase_ternaria')])))

In [ ]:
# table(round(xgb_val_preds, 3))

Aplico el modelo generado que esta en la variable "modelo" a los datos "dapply"


In [70]:
prediccion_modelo  <- predict( modelo, dtrain_imputed_cleaned_scaled, type = "prob") # aplico el modelo
# prediccion  <- predict( model_rf, dapply) # aplico el modelo
# tt = xgb.DMatrix(as.matrix(dapply_imputed))
# prediccion <- predict(bstSparse, tt) # aplico el modelo

prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2" y "CONTINUA" cada columna es el vector de probabilidades


In [71]:
dapply[ , prob_baja2 := prediccion_modelo[, "BAJA+2"] ]
dapply[ , Predicted  := as.numeric(prob_baja2 > 0.025) ]

In [72]:
entrega  <- dapply[   , list(numero_de_cliente, Predicted) ] #genero la salida

#genero el archivo para Kaggle
fwrite( entrega, file="./kaggle/K101_002_first_model.csv", sep="," )